In [1]:
# External
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
# Std
import os
# Local
from lib.functions import show_sample_images
from CGAN import CGanNet

In [2]:
NET_NAME = "Last_CGAN"
BATCH_SIZE = 32
IMAGE_WIDTH = 8
IMAGE_HEIGHT = 8
LEARNING_RATE_DISC: float = 0.0002
LEARNING_RATE_GAN: float = 0.0002
DROPOUT_RATE: float = 0.4
DENSE_UNITS=128
CONV_LAYERS = 2
BATCH_NORM = False

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
# Load flower dataset
(training_set, validation_set), dataset_info = tfds.load(
    'oxford_flowers102',
    split=['test[:50%]', 'validation'],
    with_info=True,
    as_supervised=True,
)
# Get size of datasets from dataset_info
dataset_size = dataset_info.splits['test[:50%]'].num_examples
print(f'Flower dataset size is: {dataset_size}')

Flower dataset size is: 3074


In [5]:
# Preprocessing images
def format_image(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.image.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))

    return image, label
train_batches = training_set.shuffle(dataset_size//4).map(format_image)

# Get labels numbers to names map
labels_strings = {
    1: "Kwiat",
    2: "Ptak",
    3: "Pies"
}

In [6]:
# Create numpy arrays with images and labels
numpy_train_batches_images = np.array([_[0] for _ in train_batches])
numpy_train_batches_labels = np.array([1 for _ in train_batches])

In [7]:
cgan = CGanNet(net_name=NET_NAME,
               batch_size=BATCH_SIZE,
               image_width=IMAGE_WIDTH,
               image_height=IMAGE_HEIGHT,
               learning_rate_disc= LEARNING_RATE_DISC,
               learning_rate_gan=LEARNING_RATE_GAN,
               dropout_rate= DROPOUT_RATE,
               generator_dense_units=DENSE_UNITS,
               num_conv_layers=CONV_LAYERS,
               batch_norm=BATCH_NORM,
               number_of_channels=3,
               latent_dimension=100,
               training_data=numpy_train_batches_images,
               labels_data=numpy_train_batches_labels,
               number_of_classes=3,
               labels_names = labels_strings)

In [8]:
# Define GAN models
cgan.define_discriminator()
cgan.define_generator()
cgan.define_gan()

In [9]:
cgan.train(number_of_epochs=1, load_past_model=False)

Number of images in dataset: 3074
Batches per epoch: 96
Half batch size is: 16
---> Archive old networks data: /home/kamil/Repositories/GAN_mg/Last_CGAN_data
------------------------------------------------------------
---> Epoch: 1 1/1
---> Start time is: 16:12:52
96/96 [==============================] - 8s 55ms/step
---> End time is: 16:13:00
---> D_real_loss: 0.23686115443706512 D_fake_loss: 0.5127718448638916 G_loss: 1.1098108291625977
----> D_real_acc: 1.0 D_fake_acc: 0.75
----> Epoch training time: 0.0h 0.0m, 7s


KeyboardInterrupt: 

In [ ]:
show_sample_images(numpy_train_batches_images, numpy_train_batches_labels, labels_strings, NET_NAME, dataset_size=dataset_size)

In [ ]:
cgan.visualize_models()

In [ ]:
cgan.train(number_of_epochs=100, load_past_model=True)

In [ ]:
cgan.get_training_time()